<img src="./images/banner.png" width="800">

# Training Neural Networks

Training a neural network effectively requires a clear understanding of its objectives, potential pitfalls, and the importance of proper data handling. Before diving into gradient descent, loss functions, and parameter updates, it’s crucial to frame *why* we train neural networks in a particular way and *how* different factors influence the ultimate performance. In this section, we will explore the **goals** of training, discuss **key challenges** that often arise, and highlight **data considerations and preprocessing** techniques that can make or break your model’s success.


When we talk about training neural networks, we often refer to the process of finding optimal parameters (weights and biases) that minimize a chosen loss function. Formally, for a given dataset of $N$ samples, we aim to minimize a function:

$$
J(\mathbf{w}) = \frac{1}{N}\sum_{i=1}^{N} \mathcal{L}(f(\mathbf{x}_i; \mathbf{w}), y_i),
$$


<img src="./images/nn-loss.png" width="800">

where:
- $\mathbf{x}_i$ is the input vector for the $i$th sample,  
- $y_i$ is the corresponding ground-truth label or target,  
- $f(\mathbf{x}_i; \mathbf{w})$ is the neural network’s prediction,  
- $\mathcal{L}$ is the loss function measuring the discrepancy between prediction and target,  
- $\mathbf{w}$ represents all trainable parameters (weights, biases).


Ultimately, *training* is about driving this loss function to a sufficiently low value so that the network **generalizes** well to data it hasn’t seen during training.


❗️ **Important Note:** While a low loss on the training set is good, the *real* goal is to ensure strong performance on *unseen* data, emphasizing the critical concept of *generalization*.


Training deep neural networks is not always straightforward. Several challenges can arise:

1. **Overfitting and Underfitting**  
   Overfitting happens when the model performs exceptionally well on training data but fails to generalize. Underfitting, on the other hand, occurs when the model is too simple, leading to high bias and poor training performance.

2. **Computational Resources**  
   Depending on network size and dataset complexity, training can be computationally expensive. GPUs (or TPUs) often become necessary for practical deep learning tasks.

3. **Performance Bottlenecks**  
   Inefficient code, suboptimal hardware usage, and large memory footprints can all slow down training. Techniques like mini-batching and optimized linear algebra libraries help overcome these bottlenecks.

4. **Hyperparameter Tuning**  
   Neural networks have numerous hyperparameters—learning rate, batch size, number of layers, etc. Finding the “best” combination requires systematic experimentation and validation.


💡 **Tip:** Start with simpler architectures and smaller experiments. Gradually scale up once you have verified that your setup, data pipeline, and basic hyperparameters work as intended.


The quality of your data and the way it’s prepared can significantly influence how your neural network learns:

- **Data Cleaning**  
  Remove outliers or erroneous entries that can distort the learning process.

- **Normalization and Standardization**  
  Scale input features to comparable ranges (e.g., using min-max scaling or z-score standardization). This helps accelerate convergence during training.

- **Data Augmentation**  
  In tasks like computer vision or speech recognition, augmenting data with transformations (e.g., rotations, flips, noise injection) can improve the model’s robustness.

- **Splitting Data**  
  Reserve separate *training*, *validation*, and *test* sets to ensure unbiased assessments of model performance.


In summary, the *Introduction* section underscores why we train neural networks and the various factors—from model complexity to data preparation—that impact the training process. Having a firm grasp of these basics will pave the way for effective training techniques, which we will explore in detail through the rest of this lecture.

**Table of contents**<a id='toc0_'></a>    
- [The Training Loop](#toc1_)    
  - [Iterations, Epochs, and Batches](#toc1_1_)    
  - [Forward Pass and Loss Computation](#toc1_2_)    
  - [Backpropagation Recap](#toc1_3_)    
  - [Parameter Updates](#toc1_4_)    
- [Gradient Descent Variants](#toc2_)    
  - [Overview of Optimizers](#toc2_1_)    
  - [SGD with Momentum](#toc2_2_)    
  - [Adam, RMSProp, and Others](#toc2_3_)    
- [Validation and Early Stopping](#toc3_)    
  - [Train-Validation Split](#toc3_1_)    
  - [Overfitting Indicators](#toc3_2_)    
  - [Early Stopping Criteria](#toc3_3_)    
  - [Practical Example](#toc3_4_)    
- [Practical Tips and Techniques](#toc4_)    
  - [Initialization Strategies](#toc4_1_)    
  - [Data Augmentation](#toc4_2_)    
  - [Monitoring Training Progress](#toc4_3_)    
  - [Handling Real-World Constraints](#toc4_4_)    
- [Summary](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[The Training Loop](#toc0_)

The **training loop** is the central mechanism by which neural networks update their parameters and gradually reduce their loss function. At a high level, the loop is quite straightforward: we pass data through the network to obtain predictions, measure how far those predictions deviate from the ground truth, and then adjust the network’s parameters to minimize future errors. Understanding this process is crucial to diagnosing issues like slow convergence or overfitting, and helps you maintain a clear grasp on how your model is evolving through each iteration of training.


### <a id='toc1_1_'></a>[Iterations, Epochs, and Batches](#toc0_)


When training is discussed, you’ll often hear three terms: **iterations**, **epochs**, and **batches**.

- **Iteration**: A single update of the model’s parameters.
- **Epoch**: One complete pass through the entire training dataset.
- **Batch**: A subset of the training data processed in a single iteration.


<img src="./images/iteration-epoch-batch.ppm" width="800">

Using batches is practical because processing the entire dataset at once (known as *batch* or *full-batch* gradient descent) can be computationally expensive and slow to converge. Instead, we typically use **mini-batch** gradient descent, where we load small chunks (batches) of data at a time. This approach strikes a balance between stability and speed.


For instance, suppose you have a dataset of 10,000 images and set your batch size to 100. Training on one epoch will require:

$$
\frac{10,000 \text{ images}}{100 \text{ images/batch}} = 100 \text{ iterations}
$$


Each iteration processes 100 images and updates the network’s parameters once, and after 100 iterations, you’ve completed 1 epoch.


💡 **Tip:** Try different batch sizes. *Very small* batches can lead to noisy gradient estimates, while *very large* batches can cause slow updates and high memory usage.


### <a id='toc1_2_'></a>[Forward Pass and Loss Computation](#toc0_)


The **forward pass** is the step in which input data moves through the layers of the network, ultimately producing an output (e.g., a probability distribution over classes or a continuous value for regression tasks). Mathematically, for a single data point $\mathbf{x}$, the network output can be expressed as:

$$
\hat{y} = f(\mathbf{x}; \mathbf{w}),
$$

where $\mathbf{w}$ represents all the weights and biases in the network, and $f$ denotes the network’s operations (layer-by-layer transformations, activation functions, etc.).


Once the forward pass is complete, we compare $\hat{y}$ (the prediction) to the ground truth $y$ using a **loss function** (e.g., Mean Squared Error, Cross-Entropy):

$$
\mathcal{L}(y, \hat{y}) = \text{Loss function measuring how “wrong” } \hat{y} \text{ is.}
$$


This scalar loss value acts as our signal for the backpropagation step, guiding how we update the parameters in the next iteration.


### <a id='toc1_3_'></a>[Backpropagation Recap](#toc0_)


During **backpropagation**, we calculate the gradient of the loss function with respect to each parameter in the network. This process involves:
1. **Applying the chain rule**: We break down how changes in each parameter affect the final loss.
2. **Computing partial derivatives**: For each weight $w$, we compute $\frac{\partial \mathcal{L}}{\partial w}$.
3. **Accumulating gradients**: Summing across all samples in the mini-batch to update each parameter effectively.


Although backpropagation can be mathematically intense, modern frameworks (like PyTorch or TensorFlow) handle these details automatically through *automatic differentiation*. The essential takeaway is that backpropagation provides the *direction* we should move each parameter to minimize loss.


❗️ **Important Note:** While automatic differentiation simplifies life, it’s mighty useful to have a conceptual understanding of backpropagation to troubleshoot issues such as vanishing or exploding gradients.


### <a id='toc1_4_'></a>[Parameter Updates](#toc0_)


Finally, we use the computed gradients to *update* the network’s weights and biases. In its simplest form, **vanilla gradient descent** updates each parameter $w_j$ by moving in the opposite direction of the gradient:

$$
w_j := w_j - \eta \frac{\partial \mathcal{L}}{\partial w_j},
$$


where $\eta$ is the **learning rate**, controlling the size of each update. Choosing a proper learning rate is critical: too large, and your training might diverge; too small, and your model might learn slowly or get stuck in a suboptimal region.


Below is a simplified pseudocode for the training loop using mini-batch gradient descent:


```python
# Pseudocode for a basic training loop
for epoch in range(num_epochs):
    shuffle(training_data)  # Shuffle the dataset each epoch
    for batch in get_mini_batches(training_data, batch_size):
        x_batch, y_batch = batch

        # Forward pass: compute predictions
        y_hat = model.forward(x_batch)

        # Compute loss
        loss = loss_function(y_hat, y_batch)

        # Backpropagation: compute gradients
        gradients = model.backward(loss)

        # Update parameters
        optimizer.step(model.parameters, gradients)

    # Optionally evaluate on validation data
    val_loss = validate(model, validation_data)
    print(f"Epoch {epoch}, Validation Loss: {val_loss:.4f}")
```


In practice, additional techniques—like momentum, adaptive learning rates, or more advanced optimizers—are applied to refine the update step. We will explore these optimizers in a subsequent section. For now, it’s enough to grasp that the parameter update step is the “learning” part of machine learning, translating mathematical gradients into improved performance on future forward passes.


*The Training Loop* is your neural network’s beating heart. Mastering the concepts of iterations, the forward pass, backpropagation, and updates will enable you to diagnose problems and fine-tune your model for better performance and faster convergence.

## <a id='toc2_'></a>[Gradient Descent Variants](#toc0_)

Training neural networks involves iteratively updating parameters to minimize a loss function. While **vanilla gradient descent** provides a solid baseline, more sophisticated techniques have been developed to speed up convergence and improve model performance. These *variants* of gradient descent often adapt learning rates, incorporate momentum, or adjust the gradient based on recent updates. Understanding these optimizers empowers you to fine-tune your training process effectively.


<img src="./images/batch-mini-sg.webp" width="800">

<img src="./images/batch-min-sg-2.png" width="800">

### <a id='toc2_1_'></a>[Overview of Optimizers](#toc0_)


Before diving into specific optimizers, let’s clarify why they’re necessary. Traditional gradient descent uses a *fixed* learning rate $\eta$:

$$
w := w - \eta \nabla_w \mathcal{L},
$$


where $\nabla_w \mathcal{L}$ is the gradient of the loss function $\mathcal{L}$ with respect to parameter $w$. However, real-world datasets and multi-layered architectures often present challenges such as:

- **Plateaus and Saddle Points**: Areas where gradients are nearly zero, slowing learning to a crawl.  
- **High Variance in Gradients**: Different parameters might need different rates of adjustment.  
- **Sensitive Hyperparameters**: Even slight changes in $\eta$ can cause convergence issues or divergences.


💡 **Tip:** Experiment with multiple optimizers when tackling a new dataset. Sometimes, a simple algorithm like **SGD + Momentum** can outperform more advanced methods on certain tasks.


### <a id='toc2_2_'></a>[SGD with Momentum](#toc0_)


**Stochastic Gradient Descent (SGD)** processes batches of data to update parameters incrementally. A common enhancement to SGD is **Momentum**, which smooths out updates by accumulating an exponential moving average of past gradients. Conceptually, momentum adds a fraction $\beta$ of the *previous* update to the *current* update:

$$
v := \beta v + \eta \nabla_w \mathcal{L}, \quad w := w - v,
$$

where $v$ is the velocity vector that accumulates gradients over time, and $0 \leq \beta < 1$ dictates how much “history” to retain.


<img src="./images/sgd-momentum.png" width="800">

Key benefits of momentum:
- **Overcoming Local Minima**: Smoother updates help the model escape shallow or narrow local minima.  
- **Faster Convergence**: By building “velocity,” the updates move in a more *consistently downward* direction on the loss surface.


Pseudocode for SGD with Momentum:
```python
# Assume you have gradients for w each iteration
if not velocity_initialized:
    velocity = 0

velocity = beta * velocity + (learning_rate * grad_w)
w = w - velocity
```


In practice, a typical value for $\beta$ is around 0.9, though you may need to adjust it for specific tasks.


### <a id='toc2_3_'></a>[Adam, RMSProp, and Others](#toc0_)


While momentum helps tackle many challenges, modern neural network training often relies on optimizers like **Adam** or **RMSProp**, which adapt learning rates for each parameter dynamically.

1. **RMSProp**  
   RMSProp maintains an exponential moving average of **squared gradients** to scale updates proportionally. If a parameter has consistently large gradients, RMSProp lowers the step size for that parameter, and vice versa.

   $$
   E[g^2] := \alpha E[g^2] + (1 - \alpha) g^2,\quad
   w := w - \frac{\eta}{\sqrt{E[g^2]} + \epsilon} g,
   $$

   where $g$ is the gradient, $E[g^2]$ is the running average of squared gradients, and $\alpha$ is typically set to around 0.9.

2. **Adam** (Adaptive Moment Estimation)  
   Adam combines ideas from both momentum and RMSProp. It maintains separate moving averages of **gradients** and **squared gradients**, then uses these to adaptively tweak the learning rate for each parameter. Adam is often a good default choice because it typically converges faster and requires less tuning:

   $$
   m := \beta_1 m + (1 - \beta_1) g,\quad
   v := \beta_2 v + (1 - \beta_2) g^2,\quad
   w := w - \eta \frac{m / (1-\beta_1^t)}{\sqrt{v / (1-\beta_2^t)} + \epsilon},
   $$

   where $m$ and $v$ are the first and second moment estimates, respectively, and $\beta_1, \beta_2$ control their decay rates.

3. **Other Optimizers**  
   - **Adagrad**: Adapts learning rates based on the frequency of updates, favoring rare parameters.  
   - **Adadelta**: An extension of Adagrad that seeks to reduce its aggressive, monotonically decreasing learning rate.  
   - **Nesterov Accelerated Gradient (NAG)**: A momentum variant that looks ahead before updating parameters.


<img src="./images/optimizers-performance.webp" width="600">

❗️ **Important Note:** No single optimizer is always superior. Factors like dataset size, network architecture, and hyperparameter choices significantly influence performance. Conduct small-scale experiments to find the best fit for your problem.


Leveraging these *gradient descent variants* can lead to more stable training and faster convergence. As you progress, try switching optimizers for the same task and monitor how the loss evolves. Small tweaks—like selecting the right optimizer or adjusting its hyperparameters—can make a huge difference in achieving optimal performance.

## <a id='toc3_'></a>[Validation and Early Stopping](#toc0_)

Ensuring that a neural network not only learns the provided training data but also *generalizes* to unseen data is a core goal of machine learning. **Validation** strategies and **early stopping** mechanisms help you track your model’s performance during training, identify overfitting trends, and decide when to halt the training process. In this section, we’ll cover how to properly split data, spot overfitting, and implement techniques to prevent your model from memorizing the training set at the expense of real-world accuracy.


### <a id='toc3_1_'></a>[Train-Validation Split](#toc0_)


A critical step in building reliable neural networks is splitting your dataset into distinct subsets for **training**, **validation**, and **testing**:

- **Training Set**: Used for fitting the model’s parameters (weights, biases).
- **Validation Set**: Helps tune hyperparameters and monitor overfitting.
- **Test Set**: Used at the *end* of model development to estimate out-of-sample performance.


By evaluating on the validation set periodically, you can check how well your model performs on data it hasn’t seen during training. If validation accuracy starts to diverge from training accuracy, it’s a red flag that the model may be overfitting.


❗️ **Important Note:** It’s best practice to avoid reusing the test set for validation or model selection. Repeatedly checking the test set can lead to biased performance estimates.


### <a id='toc3_2_'></a>[Overfitting Indicators](#toc0_)


**Overfitting** occurs when a neural network becomes too closely fitted on the training data, capturing noise or random fluctuations rather than generalizable patterns. Common indicators include:

1. **Rising Validation Loss**: While the training loss continues to decrease, the validation loss starts rising.
2. **Accuracy Discrepancy**: Training accuracy becomes significantly higher than validation (or test) accuracy.
3. **Erratic Model Behavior**: The model may behave unpredictably on real-world data despite impressive training metrics.


An example of tracking losses over epochs might look like this:

| Epoch | Training Loss | Validation Loss |
|-------|---------------|-----------------|
|     1 |    0.85       |     0.90        |
|     2 |    0.60       |     0.68        |
|     3 |    0.45       |     0.60        |
|     4 |    0.35       |     0.65  ← starts to rise |

Here, the training loss keeps going down, but at epoch 4, the validation loss has begun to climb. This is a classic sign of overfitting.


### <a id='toc3_3_'></a>[Early Stopping Criteria](#toc0_)


**Early stopping** is a straightforward yet powerful method to reduce overfitting. Instead of training for a fixed number of epochs, you monitor validation metrics and **stop** training once those metrics no longer improve. The idea is to prevent the model from *memorizing* the training set.


A common procedure is:
1. Choose a **patience** parameter (e.g., 5 epochs).
2. Track the validation loss (or accuracy) at each epoch.
3. If the validation loss doesn’t improve for *patience* consecutive epochs, halt training and revert to the best model checkpoint.


Mathematically, if $L_{\text{val}}(e)$ is the validation loss at epoch $e$, and $\min L_{\text{val}}$ is its best value so far, early stopping triggers when:

$$
L_{\text{val}}(e) \geq \min L_{\text{val}} + \delta \quad \text{for } \text{patience consecutive epochs},
$$

where $\delta$ can be a small threshold to ignore negligible fluctuations.


### <a id='toc3_4_'></a>[Practical Example](#toc0_)

Many deep learning frameworks provide built-in or easy-to-use callbacks for early stopping. Below is a simplified Python-style pseudocode illustrating how you might implement a train-validation loop with early stopping:


```python
best_val_loss = float('inf')
epochs_without_improvement = 0
patience = 5

for epoch in range(max_epochs):
    train_one_epoch(model, train_loader)  # Perform forward, backward, parameter updates
    val_loss = evaluate(model, val_loader)  # Compute validation loss

    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(model, "best_model_checkpoint.pth")
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement == patience:
            print("Early stopping triggered. Restoring best model...")
            model = load_model("best_model_checkpoint.pth")
            break
```


In this example:
- We store the **best validation loss** observed so far.
- If the current epoch’s validation loss improves on the best seen so far, we *reset* our patience counter and save the model checkpoint.
- If it fails to improve for a fixed number of **patience** epochs, we *stop* early and restore the best-performing model.


💡 **Tip:** Early stopping not only helps in better generalization but also reduces computational costs by halting training once additional epochs yield diminishing returns.


By carefully splitting your data, watching for signs of overfitting, and employing strategies like early stopping, you can build neural networks that *generalize* effectively. These techniques form the foundation of robust experiment management and serve as safeguards against overly optimistic training performance.

## <a id='toc4_'></a>[Practical Tips and Techniques](#toc0_)

Training a neural network involves more than just plugging data into an optimizer. Real-world scenarios can introduce complexities such as poor parameter initialization, noisy or limited data, and hardware constraints. In this section, we’ll explore best practices that help accelerate training, improve generalization, and enable better monitoring of your model’s performance.


### <a id='toc4_1_'></a>[Initialization Strategies](#toc0_)


Choosing an effective way to initialize network weights can make a noticeable difference in training speed and stability. If weights are initialized poorly—say, all to zero or very large random values—your model may fail to learn effectively.


- **Xavier/Glorot Initialization**
  Designed for layers with sigmoid or tanh activations, this method scales weights based on the number of incoming and outgoing connections to maintain signal variance. Formally, weights $w$ are sampled from a distribution like:

  $$
  w \sim \mathcal{N}\Big(0, \frac{2}{n_{\text{in}} + n_{\text{out}}}\Big).
  $$


- **He Initialization**
  Suited for ReLU and variants, it uses:

  $$
  w \sim \mathcal{N}\Big(0, \frac{2}{n_{\text{in}}}\Big),
  $$

  enabling gradients to flow without vanishing or exploding too quickly.


❗️ **Important Note:** While most deep-learning frameworks provide sensible defaults (e.g., Keras initializes with Glorot by default), understanding these strategies helps you tweak settings for complex architectures.


### <a id='toc4_2_'></a>[Data Augmentation](#toc0_)


Data augmentation artificially increases the size and diversity of your dataset, making your model more robust to variations. This is especially vital in fields like computer vision or speech recognition, where collecting large labeled datasets can be expensive or impractical.

- **Computer Vision Examples**: Random cropping, flipping, rotating, or adjusting brightness of images.
- **Text/NLP Examples**: Synonym replacements, random insertions, or back-translation of text.
- **Audio Examples**: Time-shifting, adding background noise, or changing pitch.


<img src="./images/data-agumentation.webp" width="800">

<img src="./images/data-agumentation-text.jpg" width="400">

Below is a simple example in PyTorch for augmenting images:


```python
import torchvision.transforms as T

transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ToTensor()
])
```


By applying these manipulations, you can help your model learn more general features and reduce overfitting.


### <a id='toc4_3_'></a>[Monitoring Training Progress](#toc0_)


Regularly tracking key metrics during training allows you to identify potential problems early and gauge when to stop or adjust hyperparameters.

- **Loss Curves**: Plot both training and validation loss over epochs. A growing gap often signals overfitting.
- **Metrics Dashboards**: Tools like TensorBoard, Weights & Biases, or MLflow can provide real-time visualizations and logs of accuracy, parameter histograms, and more.
- **Intermediate Validation**: Occasionally evaluate performance on a validation set to track generalization closely.


<img src="./images/observability-tools.jpg" width="800">

💡 **Tip:** Automate logging and checkpoint saving. This way, you can resume training from a stable point if your process is interrupted or if you decide to revisit a previously successful state.


### <a id='toc4_4_'></a>[Handling Real-World Constraints](#toc0_)

Academic examples often assume unlimited training time and perfectly clean data. In reality, you may face:

- **Limited Computation Resources**: Optimize code by using vectorized operations, mixed-precision training, or smaller mini-batches that fit your GPU memory.
- **Sparse or Noisy Data**: Use domain-specific preprocessing, regularization, or robust loss functions (e.g., Huber loss) to handle outliers and noisy labels.
- **Latency Requirements**: For real-time applications (like autonomous vehicles or high-frequency trading), model size and inference speed can be as critical as accuracy.


When constraints are tight, consider simpler architectures or specialized hardware accelerators (like GPUs with Tensor Cores or TPUs). Striking a balance between accuracy, speed, and resource usage is key in production environments.


Putting these practical tips and techniques into action can drastically enhance your training experience. From smart initialization and data augmentation to monitoring and dealing with real-world limitations, each strategy contributes to building robust, **well-trained** neural networks that are poised for success in real applications.

## <a id='toc5_'></a>[Summary](#toc0_)

This lecture covered the core elements of **training neural networks**, from setting clear goals and understanding the iterative nature of updates to leveraging validation strategies and practical techniques. By now, you should have a solid grasp of how data, optimization methods, and real-world constraints intertwine during the training process.


Neural network training revolves around minimizing a *loss function* by iteratively adjusting weights and biases. The **training loop** plays a central role, encompassing the forward pass, loss calculation, backpropagation, and parameter updates. While basic gradient descent provides a strong foundation, **variants like SGD with Momentum, RMSProp, and Adam** can significantly improve convergence speed and stability.  
Validation techniques—especially **early stopping**—help ensure that your model generalizes rather than memorizes. Meanwhile, incorporating robust **initialization strategies, data augmentation, and careful monitoring** can keep training on track and improve real-world performance.


With the fundamentals of training neural networks in place, you can explore deeper or more specialized topics such as **advanced optimizers**, **regularization** (e.g., dropout, batch normalization), and **complex architectures** (e.g., convolutional or recurrent networks). As you continue experimenting and refining your workflow, make sure to balance theoretical knowledge with hands-on practice. Staying up to date with improvements in tools and libraries will also help you train models more efficiently and effectively.


By applying these strategies and best practices, you’ll be well-equipped to develop robust neural networks that perform reliably—even under challenging real-world conditions.